In [1]:
%load_ext Cython

In [2]:
%%cython

import numpy as np
cimport numpy as np
import cython

In [3]:
# Situation setup
def f(x):
    return (x**2) - 0.8 * np.cos(30*x)

U_interval = (-1, 1)
n_strategies = 100
U = np.concatenate((
    np.linspace(
        U_interval[0], 0, n_strategies//2, endpoint=False),
    np.linspace(
        0, U_interval[1], n_strategies//2 + 1, endpoint=True))
)[:, None]

point_interval = (-10, 10)
n_points = 200
points = np.random.uniform(
    point_interval[0], point_interval[1], n_points)
N = len(points)
d = len(points[0]) if isinstance(points[0], tuple) else 1
locations = np.array(points).reshape(N, d)

alpha = 2

In [4]:
# Original code
def original_naive(x, u, x2):
    if x==x2:
        if u==0:
            return 1
        else:
            return 0

    with np.errstate(divide='ignore'):
        out = np.exp(
            -((u - (max(0, np.tanh(3*(f(x) - f(x2)))) *
                    (x2 - x)))**2) /
            (np.abs(x-x2) ** alpha))

    return out

In [5]:
%%timeit
original_naive(locations[0], U[0], locations[1])

41 µs ± 968 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
%%cython
import numpy as np
# Original code but compiled
def f(x):
    return (x**2) - 0.8 * np.cos(30*x)
alpha = 2
def original_compiled_naive(x, u, x2):
    if x==x2:
        if u==0:
            return 1
        else:
            return 0

    with np.errstate(divide='ignore'):
        out = np.exp(
            -((u - (max(0, np.tanh(3*(f(x) - f(x2)))) *
                    (x2 - x)))**2) /
            (np.abs(x-x2) ** alpha))

    return out

In [7]:
%%timeit
original_compiled_naive(locations[0], U[0], locations[1])

38 µs ± 357 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
%%cython
from libc.math cimport cos, exp, tanh, fabs, fmax

import numpy as np
cimport numpy as np

DTYPE = np.float64
ctypedef np.float64_t DTYPE_t

# Original code but compiled
cdef DTYPE_t f(DTYPE_t x):
    return (x**2) - 0.8 * cos(30*x)
cdef int alpha = 2

cpdef DTYPE_t cython_naive(DTYPE_t x, DTYPE_t u, DTYPE_t x2):
    if x==x2:
        if u==0:
            return 1
        else:
            return 0

    out = exp(
        -((u - (fmax(0, tanh(3*(f(x) - f(x2)))) * (x2 - x)))**2) /
        (fabs(x-x2) ** alpha))

    return out

In [9]:
%%timeit
cython_naive(locations[0], U[0], locations[1])

738 ns ± 4.17 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Now for the looping of the above function

In [32]:
%%cython --compile-args=-fopenmp --compile-args=-O3 --link-args=-fopenmp
from libc.math cimport cos, exp, tanh, fabs, fmax
 
import cython
import numpy as np
cimport numpy as np
from cython.parallel import prange

DTYPE = np.float64
ctypedef np.float64_t DTYPE_t

# Original code but compiled
cpdef DTYPE_t f(DTYPE_t x) nogil:
    return (x**2) - 0.8 * cos(30*x)

cdef int alpha = 2


@cython.cdivision(True)
cdef DTYPE_t cython_naive(DTYPE_t x, DTYPE_t u, DTYPE_t x2) nogil:
    if x==x2:
        if u==0:
            return 1
        else:
            return 0
    cdef DTYPE_t out
    out = exp(
        -((u - (fmax(0, tanh(3*(f(x) - f(x2)))) * (x2 - x)))**2) /
        (fabs(x-x2) ** alpha))
    
    return out


@cython.boundscheck(False)
@cython.wraparound(False)
cpdef DTYPE_t [:,:,:] vectorized(DTYPE_t [:] locations, DTYPE_t [:] U):
    cdef int N = locations.shape[0]
    cdef int d = locations.shape[1]
    cdef int i, j, k
    cdef int U_number = U.shape[0]
    #assert d == 1, "Multi dimensional not implemented yet"
    #locations = locations.flatten()
    #U = U.flatten()
    
    cdef DTYPE_t [:,:,:] out = np.empty((N, N, U.shape[0]))
    #cdef DTYPE_t out[N][N][U_number]

    for k in prange(U_number, nogil=True):
        for i in range(N):
            for j in range(N):
                out[i, j, k] = cython_naive(
                    locations[i], U[k], locations[j])
    return out

In [33]:
%%timeit
vectorized(locations.flatten(), U.flatten())

189 ms ± 2.07 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Reference: My vectorized implementation

In [30]:
def _vectorized(locations, U):
        """Idea: generate a whole NxNx#Strategies tensor with the values of J

        This one is actually used for computations.

        axis=0 the point to evaluate
        axis=1 the point to compare to
        axis=2 are the strategies
        """
        N, d = locations.shape

        f_vals = f(locations)
        assert len(f_vals) == N, 'f is not suited for higher-dimensional stuff'
        f_vals = f_vals.flatten()
        f_diffs = np.tile(f_vals, reps=(N, 1)).T - np.tile(f_vals, reps=(N, 1))
        f_diffs_tanh = np.tanh(3*f_diffs)
        f_diffs_tanh_positive = np.where(
            f_diffs_tanh > 0,
            f_diffs_tanh,
            0)
        # f_diffs_tanh_positive = sigmoid(100*f_diffs)

        # Walk dirs should be a NxNxd array, containing xj-xi at location [i, j, :]
        walk_dirs = (np.tile(locations[None, :, :], (N, 1, 1)) -
                     np.tile(locations[:, None, :], (1, N, 1)))

        walk_dirs_adj = f_diffs_tanh_positive[:, :, None] * walk_dirs

        ##############################################################
        # Multi dimensionality does not yet work, starting from here #
        ##############################################################
        walk_dirs_adj = walk_dirs_adj.reshape(N, N)
        walk_dirs = walk_dirs.reshape(N, N)
        variance = (np.abs(walk_dirs) ** alpha +
                    np.abs(f_diffs) ** alpha)
        # variance = (np.abs(walk_dirs) ** self.alpha)

        # Make things stable for x=x2
        problems = np.array([
            [np.all(locations[i] == locations[j]) for j in range(N)]
            for i in range(N)])
        variance[problems] = 1

        upper_side = (U.flatten()[None, None, :] -
                      walk_dirs_adj[:, :, None]) ** 2

        out = np.exp(-1 * upper_side / variance[:, :, None])

        # Set the "problems" to the values they should contain (by analysis of J)
        out[problems] = 0
        standing_index = np.where(U==0)[0][0]
        out[problems, standing_index] = 1

        return out

In [31]:
%%timeit
_vectorized(locations, U)

430 ms ± 2.99 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
